## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'
sheet_value = 'TOP VALOR'
sheet_packs = 'TOP PACKS'
sheet_services = 'TOP SERVICIOS'

# Range to read (change this for each project)
start_row = 8
end_row = 150
usecols = 'A:AF'

print("Variables defined correctly!")

Variables defined correctly!


## Verify Columns in 'Active' Sheet.

Verify the number of columns in the "Active" sheet to ensure the range is within bounds. We will use the range in all three sheets

In [3]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

Max column in 'Active' sheet: 21
Adjusted usecols to: A:U


## Extract Data from all six Sheets

Extract data from all three Sheets within the specified range and convert it directly to a DataFrame.

In [4]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)
value_df = load_sheet_as_dataframe(excel_file_path, 'TOP VALOR', start_row, end_row, usecols)
packs_df = load_sheet_as_dataframe(excel_file_path, 'TOP PACKS', start_row, end_row, usecols)
services_df = load_sheet_as_dataframe(excel_file_path, 'TOP SERVICIOS', start_row, end_row, usecols)

# Display the DataFrames
print("Active DataFrame:")
display(active_df.head(99))

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

print("value DataFrame:")
display(value_df.head())

print("Packs DataFrame:")
display(packs_df.head())

print("Services DataFrame:")
display(services_df.head())


Data from 'ACTIVOS' sheet loaded successfully.
Data from 'CANCELADOS' sheet loaded successfully.
Data from 'PTE FIRMA' sheet loaded successfully.
Data from 'TOP VALOR' sheet loaded successfully.
Data from 'TOP PACKS' sheet loaded successfully.
Data from 'TOP SERVICIOS' sheet loaded successfully.
Active DataFrame:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,22.0,4.0,4.0,11.0,...,NaN,8.0,16.0,6.0,4.0,19.0,NaN,NaN,6.0,123
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,16.0,10.0,11.0,...,NaN,12.0,9.0,22.0,5.0,21.0,NaN,NaN,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,NaN,NaN,10.0,15.0,30.0,23.0,11.0,...,NaN,13.0,20.0,14.0,10.0,19.0,NaN,NaN,30.0,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Fibra 600 con permanencia.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
84,victor.justamante@finetwork.com,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,4
85,yicel.patricia@originaltelecom.es,NaN,NaN,16.0,7.0,9.0,NaN,NaN,NaN,NaN,...,12.0,7.0,NaN,NaN,NaN,NaN,27.0,24.0,11.0,132
86,yzabelly.gomes@originaltelecom.es,NaN,NaN,14.0,15.0,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,30.0,18.0,NaN,89


Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,36,183,111,71,47,38,29,12,33,1539
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,2.0,NaN,NaN,NaN,3.0,1.0,5.0,3.0,10.0,...,NaN,4.0,1.0,NaN,NaN,7.0,NaN,NaN,NaN,36
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,albertocanto@originaltelecom.es,1.0,5.0,NaN,NaN,NaN,5.0,5.0,3.0,4.0,...,NaN,4.0,3.0,NaN,NaN,4.0,NaN,NaN,NaN,34
4,albertosanchez@originaltelecom.es,3.0,1.0,NaN,NaN,3.0,3.0,6.0,NaN,3.0,...,NaN,3.0,6.0,2.0,2.0,4.0,NaN,NaN,NaN,36


Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,219,189,104,64,262,245,227,195,208,...,75,246,261,229,135,265,139,96,388,3633
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,3.0,4.0,NaN,NaN,1.0,16.0,2.0,4.0,6.0,...,NaN,5.0,8.0,5.0,4.0,17.0,NaN,NaN,6.0,81
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,albertocanto@originaltelecom.es,4.0,7.0,NaN,NaN,11.0,4.0,7.0,7.0,7.0,...,NaN,7.0,6.0,10.0,4.0,14.0,NaN,NaN,10.0,98
4,albertosanchez@originaltelecom.es,11.0,6.0,NaN,NaN,6.0,15.0,17.0,12.0,7.0,...,NaN,8.0,16.0,7.0,4.0,16.0,NaN,NaN,21.0,146


Packs DataFrame:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,77,286.1,305,296,168,299,156,124.1,510,4359
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,6.0,5.0,NaN,NaN,5.0,19.0,3.0,3.0,11.0,...,NaN,6.0,13.0,3.0,3.0,15.0,NaN,NaN,3.0,95
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3
3,albertocanto@originaltelecom.es,7.0,6.0,NaN,NaN,6.0,3.0,15.0,10.0,11.0,...,NaN,9.0,5.0,22.0,3.0,18.0,NaN,NaN,15.0,130
4,albertosanchez@originaltelecom.es,12.0,8.0,NaN,NaN,5.0,7.0,25.0,21.0,9.0,...,NaN,8.0,13.0,10.0,10.0,15.0,NaN,NaN,26.0,169


Services DataFrame:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,22.0,4.0,4.0,11.0,...,NaN,8.0,16.0,6.0,4.0,19.0,NaN,NaN,6.0,123
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,16.0,10.0,11.0,...,NaN,12.0,9.0,22.0,5.0,21.0,NaN,NaN,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,NaN,NaN,10.0,15.0,30.0,23.0,11.0,...,NaN,13.0,20.0,14.0,10.0,19.0,NaN,NaN,30.0,220


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [5]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)
value_df = replace_nan_with_zero(value_df)
packs_df = replace_nan_with_zero(packs_df)
services_df = replace_nan_with_zero(services_df)



# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head(88))

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

print("Value DataFrame after replacing NaN:")
display(value_df.head())

print("Packs DataFrame after replacing NaN:")
display(packs_df.head())

print("Services DataFrame after replacing NaN:")
display(services_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Fibra 600 con permanencia.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
84,victor.justamante@finetwork.com,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4
85,yicel.patricia@originaltelecom.es,0.0,0.0,16.0,7.0,9.0,0.0,0.0,0.0,0.0,...,12.0,7.0,0.0,0.0,0.0,0.0,27.0,24.0,11.0,132
86,yzabelly.gomes@originaltelecom.es,0.0,0.0,14.0,15.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,30.0,18.0,0.0,89


Canceled DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,36,183,111,71,47,38,29,12,33,1539
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0,10.0,...,0.0,4.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,36
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,0.0,4.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,34
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,3.0,...,0.0,3.0,6.0,2.0,2.0,4.0,0.0,0.0,0.0,36


Pending Signature DataFrame after replacing NaN:


...

,Inbound Telec.Orig.Sevilla,219,189,104,64,262,245,227,195,208,...,75,246,261,229,135,265,139,96,388,3633
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,16.0,2.0,4.0,6.0,...,0.0,5.0,8.0,5.0,4.0,17.0,0.0,0.0,6.0,81
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,0.0,7.0,6.0,10.0,4.0,14.0,0.0,0.0,10.0,98
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,7.0,...,0.0,8.0,16.0,7.0,4.0,16.0,0.0,0.0,21.0,146


Packs DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,77,286.1,305,296,168,299,156,124.1,510,4359
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,19.0,3.0,3.0,11.0,...,0.0,6.0,13.0,3.0,3.0,15.0,0.0,0.0,3.0,95
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,0.0,9.0,5.0,22.0,3.0,18.0,0.0,0.0,15.0,130
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,0.0,8.0,13.0,10.0,10.0,15.0,0.0,0.0,26.0,169


Services DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220


## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [6]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [7]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)
value_df = ensure_all_agents(value_df, agents_list)
packs_df = ensure_all_agents(packs_df, agents_list)
services_df = ensure_all_agents(services_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head(88))

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

print("Value DataFrame after ensuring all agents:")
display(value_df.head())

print("Packs DataFrame after ensuring all agents:")
display(packs_df.head())

print("Services DataFrame after ensuring all agents:")
display(services_df.head())

Added 2 missing agents and removed 45 agents not in the list.
Added 2 missing agents and removed 28 agents not in the list.
Added 10 missing agents and removed 0 agents not in the list.
Added 2 missing agents and removed 27 agents not in the list.
Added 3 missing agents and removed 40 agents not in the list.
Added 2 missing agents and removed 45 agents not in the list.
Active DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,3.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,0.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126
9,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,36.0,14.0,0.0,77
10,beatriz.gomez@originaltelecom.es,6.0,9.0,0.0,0.0,5.0,6.0,6.0,9.0,15.0,...,0.0,18.0,9.0,11.0,5.0,11.0,0.0,0.0,13.0,123
12,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,11.0,16.0,11.0,15.0,16.0,...,0.0,11.0,22.0,20.0,9.0,13.0,0.0,0.0,27.0,187
14,cesar.arnaldo@originaltelecom.es,0.0,0.0,16.0,7.0,5.0,0.0,0.0,0.0,0.0,...,4.0,9.0,0.0,0.0,0.0,0.0,23.0,9.0,16.0,106
19,david.molero@originaltelecom.es,8.0,5.0,0.0,0.0,17.0,15.0,25.0,19.0,24.0,...,0.0,25.0,29.0,16.0,6.0,0.0,0.0,0.0,24.0,213


Canceled DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,36,183,111,71,47,38,29,12,33,1539
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0,10.0,...,0.0,4.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,36
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,0.0,4.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,34
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,3.0,...,0.0,3.0,6.0,2.0,2.0,4.0,0.0,0.0,0.0,36
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,10.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,2.0,53
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,0.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,2.0,42


Pending Signature DataFrame after ensuring all agents:


...

,Inbound Telec.Orig.Sevilla,219,189,104,64,262,245,227,195,208,...,75,246,261,229,135,265,139,96,388,3633
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,16.0,2.0,4.0,6.0,...,0.0,5.0,8.0,5.0,4.0,17.0,0.0,0.0,6.0,81
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,0.0,7.0,6.0,10.0,4.0,14.0,0.0,0.0,10.0,98
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,7.0,...,0.0,8.0,16.0,7.0,4.0,16.0,0.0,0.0,21.0,146
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,3.0,13.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,79
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,0.0,9.0,7.0,4.0,5.0,0.0,0.0,0.0,11.0,83


Packs DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,77,286.1,305,296,168,299,156,124.1,510,4359
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,19.0,3.0,3.0,11.0,...,0.0,6.0,13.0,3.0,3.0,15.0,0.0,0.0,3.0,95
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,0.0,9.0,5.0,22.0,3.0,18.0,0.0,0.0,15.0,130
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,0.0,8.0,13.0,10.0,10.0,15.0,0.0,0.0,26.0,169
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,2.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,11.0,87
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,12.0,8.0,...,0.0,6.0,11.0,8.0,11.0,0.0,0.0,0.0,10.0,95


Services DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,3.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,0.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [8]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)
value_df = add_email_values(value_df, email_values)
packs_df = add_email_values(packs_df, email_values)
services_df = add_email_values(services_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

print("Value DataFrame with email values:")
display(value_df.head())

print("Packs DataFrame with email values:")
display(packs_df.head())

print("Services DataFrame with email values:")
display(services_df.head())

Active DataFrame with email values:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,362,386,375,207,384,209,146,618,5446,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220,3
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116,4
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126,5


Canceled DataFrame with email values:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,183,111,71,47,38,29,12,33,1539,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0,10.0,...,4.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,36,1
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,4.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,34,2
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,3.0,...,3.0,6.0,2.0,2.0,4.0,0.0,0.0,0.0,36,3
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,1.0,7.0,0.0,0.0,0.0,0.0,0.0,2.0,53,4
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,8.0,4.0,4.0,0.0,0.0,0.0,0.0,2.0,42,5


Pending Signature DataFrame with email values:


...

,Inbound Telec.Orig.Sevilla,219,189,104,64,262,245,227,195,208,...,246,261,229,135,265,139,96,388,3633,email_value
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,16.0,2.0,4.0,6.0,...,5.0,8.0,5.0,4.0,17.0,0.0,0.0,6.0,81,1
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,7.0,6.0,10.0,4.0,14.0,0.0,0.0,10.0,98,2
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,7.0,...,8.0,16.0,7.0,4.0,16.0,0.0,0.0,21.0,146,3
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,13.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,79,4
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,9.0,7.0,4.0,5.0,0.0,0.0,0.0,11.0,83,5


Packs DataFrame with email values:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,286.1,305,296,168,299,156,124.1,510,4359,email_value
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,19.0,3.0,3.0,11.0,...,6.0,13.0,3.0,3.0,15.0,0.0,0.0,3.0,95,1
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,9.0,5.0,22.0,3.0,18.0,0.0,0.0,15.0,130,2
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,13.0,10.0,10.0,15.0,0.0,0.0,26.0,169,3
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,9.0,2.0,0.0,0.0,0.0,0.0,0.0,11.0,87,4
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,12.0,8.0,...,6.0,11.0,8.0,11.0,0.0,0.0,0.0,10.0,95,5


Services DataFrame with email values:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,362,386,375,207,384,209,146,618,5446,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220,3
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116,4
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126,5


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [9]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)
value_df = sort_by_email_value(value_df)
packs_df = sort_by_email_value(packs_df)
services_df = sort_by_email_value(services_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head()) 

print("Sorted Canceled DataFrame:")
display(canceled_df.head())

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())

print("Sorted Canceled DataFrame:")
display(value_df.head())

print("Sorted Pending Signature DataFrame:")
display(packs_df.head())

print("Sorted Pending Signature DataFrame:")
display(services_df.head())

Sorted Active DataFrame:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,362,386,375,207,384,209,146,618,5446,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220,3
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116,4
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126,5


Sorted Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,183,111,71,47,38,29,12,33,1539,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0,10.0,...,4.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,36,1
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,4.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,34,2
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,3.0,...,3.0,6.0,2.0,2.0,4.0,0.0,0.0,0.0,36,3
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,1.0,7.0,0.0,0.0,0.0,0.0,0.0,2.0,53,4
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,8.0,4.0,4.0,0.0,0.0,0.0,0.0,2.0,42,5


Sorted Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,219,189,104,64,262,245,227,195,208,...,246,261,229,135,265,139,96,388,3633,email_value
1,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,16.0,2.0,4.0,6.0,...,5.0,8.0,5.0,4.0,17.0,0.0,0.0,6.0,81,1
3,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,7.0,6.0,10.0,4.0,14.0,0.0,0.0,10.0,98,2
4,albertosanchez@originaltelecom.es,11.0,6.0,0.0,0.0,6.0,15.0,17.0,12.0,7.0,...,8.0,16.0,7.0,4.0,16.0,0.0,0.0,21.0,146,3
5,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,13.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,79,4
6,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,8.0,...,9.0,7.0,4.0,5.0,0.0,0.0,0.0,11.0,83,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,286.1,305,296,168,299,156,124.1,510,4359,email_value
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,19.0,3.0,3.0,11.0,...,6.0,13.0,3.0,3.0,15.0,0.0,0.0,3.0,95,1
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,9.0,5.0,22.0,3.0,18.0,0.0,0.0,15.0,130,2
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,8.0,13.0,10.0,10.0,15.0,0.0,0.0,26.0,169,3
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,9.0,2.0,0.0,0.0,0.0,0.0,0.0,11.0,87,4
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,12.0,8.0,...,6.0,11.0,8.0,11.0,0.0,0.0,0.0,10.0,95,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,362,386,375,207,384,209,146,618,5446,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159,2
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220,3
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116,4
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126,5


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [10]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)
value_df = remove_email_value_column(value_df)
packs_df = remove_email_value_column(packs_df)
services_df = remove_email_value_column(services_df)


# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

print("Value DataFrame after removing 'email_value' column:")
display(value_df.head())

print("Packs DataFrame after removing 'email_value' column:")
display(packs_df.head())

print("Services DataFrame after removing 'email_value' column:")
display(services_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

Active DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,3.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,0.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126


Canceled DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,109,114,60,54,127,104,130,110,110.1,...,36,183,111,71,47,38,29,12,33,1539
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,3.0,1.0,5.0,3.0,10.0,...,0.0,4.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,36
3,albertocanto@originaltelecom.es,1.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,4.0,...,0.0,4.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,34
4,albertosanchez@originaltelecom.es,3.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,3.0,...,0.0,3.0,6.0,2.0,2.0,4.0,0.0,0.0,0.0,36
5,anasanchez@originaltelecom.es,0.0,0.0,9.0,7.0,7.0,2.0,4.0,0.0,0.0,...,10.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,2.0,53
6,antonio.reina@originaltelecom.es,2.0,7.0,0.0,0.0,3.0,1.0,1.0,0.0,10.0,...,0.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,2.0,42


Pending Signature DataFrame after removing 'email_value' column:


...

Packs DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,247,224,124,93,276,281,286,240,267,...,77,286.1,305,296,168,299,156,124.1,510,4359
1,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,19.0,3.0,3.0,11.0,...,0.0,6.0,13.0,3.0,3.0,15.0,0.0,0.0,3.0,95
3,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,15.0,10.0,11.0,...,0.0,9.0,5.0,22.0,3.0,18.0,0.0,0.0,15.0,130
4,albertosanchez@originaltelecom.es,12.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,0.0,8.0,13.0,10.0,10.0,15.0,0.0,0.0,26.0,169
5,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,2.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,11.0,87
6,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,12.0,8.0,...,0.0,6.0,11.0,8.0,11.0,0.0,0.0,0.0,10.0,95


Services DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,326,271,168,106,351,355,345,292,316,...,93,362,386,375,207,384,209,146,618,5446
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,22.0,4.0,4.0,11.0,...,0.0,8.0,16.0,6.0,4.0,19.0,0.0,0.0,6.0,123
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,16.0,10.0,11.0,...,0.0,12.0,9.0,22.0,5.0,21.0,0.0,0.0,18.0,159
4,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,30.0,23.0,11.0,...,0.0,13.0,20.0,14.0,10.0,19.0,0.0,0.0,30.0,220
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,3.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,13.0,116
8,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,18.0,10.0,...,0.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,16.0,126


Values from the second column of active_df:
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroy

## Process Data from all DataFrames

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [11]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2
value_sums = value_df.iloc[:, 1:].sum(axis=1) / 2
packs_sums = packs_df.iloc[:, 1:].sum(axis=1) / 2
services_sums = services_df.iloc[:, 1:].sum(axis=1) / 2

print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head(66))# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())
print("Calculated sums for 'Value' DataFrame:")
print(value_sums.head())
print("Calculated sums for 'Packs' DataFrame:")
print(packs_sums.head())
print("Calculated sums for 'Services' DataFrame:")
print(services_df.head())

Calculated sums for 'Active' DataFrame:
1     123.0
3     159.0
4     220.0
6     116.0
8     126.0
9      77.0
10    123.0
57    107.0
12    187.0
14    106.0
19    213.0
23    152.0
25     89.0
34    196.0
36    205.0
37    149.0
39     70.0
42      3.0
48    175.0
47    110.0
50    146.0
22    138.0
52    123.0
88      0.0
89      0.0
56    121.0
55    148.0
54     74.0
59    116.0
60     98.0
62    199.0
63    132.0
53     37.0
67    113.0
68     89.0
71     55.0
72    156.0
74     70.0
75     99.0
79     32.0
80    139.0
85    132.0
86     89.0
dtype: float64
Calculated sums for 'Canceled' DataFrame:
1    36.0
3    34.0
4    36.0
5    53.0
6    42.0
dtype: float64
Calculated sums for 'Pending Signature' DataFrame:
0    6.0
1    7.0
2    2.0
3    1.0
4    3.0
dtype: float64
Calculated sums for 'Value' DataFrame:
1     81.0
3     98.0
4    146.0
5     79.0
6     83.0
dtype: float64
Calculated sums for 'Packs' DataFrame:
1     95.0
3    130.0
4    169.0
5     87.0
6     95.0
dtype: f

## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from all five DataFrame

### Update finalFile Excel sheet with Active Data

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame in column J.

In [12]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'.


### Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column K.

In [13]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Canceled data.


### Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column L.

In [14]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Pending Signature data.


### Update finalFile Excel Sheet with Values Data

Update the "finalFile" Excel sheet with the calculated values from the Values DataFrame in column M.

In [15]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 13  # Column M
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Values data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, value_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Values data.


### Update finalFile Excel Sheet with Packs Data

Update the "finalFile" Excel sheet with the calculated values from the Packs DataFrame in column N.

In [16]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 14  # Column N
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Packs data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, packs_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Packs data.


### Update finalFile Excel Sheet with Services Data

Update the "finalFile" Excel sheet with the calculated values from the Services DataFrame in column O.

In [17]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 15  # Column O
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Services data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, services_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Services data.
